In [ ]:
from typing import Dict
import asyncio

import re
import pandas as pd
import numpy as np
from doraemon import Doraemon
from kg_rag import PromptBuilder, KG_RAG_Tool, Inference


def classify(x: str, n: int = 1200, dataset: str = "metaqa") -> pd.DataFrame:
    BASE_PATHS = {
        "metaqa": "/kaggle/input/filtered-multiple-hops-metaqa",
        "webqsp": "/kaggle/input/webqsp"
    }

    # Validate dataset input
    if dataset not in BASE_PATHS:
        raise ValueError(f"Invalid dataset '{dataset}'. Must be one of: {list(BASE_PATHS.keys())}")

    base = BASE_PATHS[dataset]
    x = (x or "").strip().lower()

    # Select file path based on dataset type
    if dataset == "metaqa":
        match x:
            case "one":
                path = f"{base}/one_hop_supported.pickle"
            case "three" | _:
                path = f"{base}/three_hop_supported.pickle"
    else:  # dataset == "another"
        match x:
            case "one":
                path = f"{base}/webqsp_ctxstyle_1200_hop1_nl.pkl"
            case "three":
                path = f"{base}/webqsp_ctxstyle_1200_hop3_nl.pkl"
            case "two" | _:
                raise ValueError("❌ 'two_hop_supported.pickle' does not exist in the 'current' dataset.")

    # Load and return dataframe
    df = pd.read_pickle(path)

    if dataset =="webqsp":
        df=df.rename(columns={
            "ground_truth":"Label",
            "contexts": "ctx_topk"
        })
    
    return df.head(n)


class Local:

    @classmethod
    def norm(cls, x):
        """
        Normalise answers for robust comparison:
        - remove backslashes
        - strip quotes/brackets/whitespace
        - lowercase
        - collapse internal whitespace
        """
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return None
        s = str(x)
        s = s.replace('\\', '')
        s = s.strip().strip('\'"[]{}()')
        s = re.sub(r'\s+', ' ', s).lower()
    
        return s
    
    @classmethod
    def eval_accuracy(
            cls,
            df: pd.DataFrame,
            pred: str,
            g_t: str,
            out_col: str = "is_correct"
        ) -> Dict[str, float]:
            """
            Evaluate accuracy by checking if prediction is inside the label list.
    
            Args:
                df (pd.DataFrame): DataFrame containing predictions and labels.
                pred (str): Column name for predictions.
                g_t (str): Column name for ground-truth labels (list-like).
                out_col (str): Column name to store correctness flags.
    
            Returns:
                dict: {"accuracy": float, "num_correct": int, "total": int}
            """
    
            def _in_labels(p, labels) -> bool:
                if p is None or (isinstance(p, float) and pd.isna(p)):
                    return False

                if isinstance(labels, np.ndarray):
                    labels = labels.tolist()
                if not isinstance(labels, (list, tuple, set)):
                    return False
    
                norm_p =cls.norm(p)
                norm_labels = {cls.norm(x) for x in labels if x is not None}
                return norm_p in norm_labels
    
            # Mark correctness per row
            df[out_col] = df.apply(lambda r: _in_labels(r[pred], r[g_t]), axis=1)
    
            total = len(df)
            num_correct = int(df[out_col].sum()) if total else 0
            accuracy = (num_correct / total) if total else float("nan")
    
            return {"accuracy": accuracy, "num_correct": num_correct, "total": total}

class StanRAG:
    SYSTEM_PROMPT: Dict[str, str]={
        'rag':(
            "You are an assistant that guess answers for the questions based on the provided context."
            "Return the final answer and a confidence score between 0.00 and 1.00"
            "in the format: \\boxed{{answer, confidence}}. "
            "where <confidence> reflects how confident you are that no correct answer exists in the context."
        )
    }
    
    @classmethod
    def build_user_content(cls, q, ctxs):
        # Ensure ctxs is always a list
        if ctxs is None:
            ctxs = []
        if not isinstance(ctxs, (list, tuple)):
            ctxs = [ctxs]
    
        # Take only the first context if available
        first_ctx = ctxs[0] if len(ctxs) > 0 else ""
    
        # Build output string
        lines = [f"Question: {q}"]
        if first_ctx:
            lines.append(f"Context1: {first_ctx}")

        lines.append("Return the final answer in \\boxed{{answer, confidence}}")
    
        return "\n".join(lines)

In [ ]:
from calibration_metrics import CalibrationMetrics

dataset="webqsp"
df = classify("three", n = 1200, dataset=dataset)    # From MetaQA dataset

df["query"] = df.apply(
    lambda row: [
        {"role": "system", "content": StanRAG.SYSTEM_PROMPT["rag"]},
        {"role": "user", "content": StanRAG.build_user_content(row["question"], row["ctx_topk"])},
    ],
    axis=1
)

Doraemon.set_provider('llama3')
logger=Doraemon.get_logger(logfile='baselines_rag.log')
tasks = df.to_dict(orient='records')

std_a = await Inference.process_batches(tasks, logger, 'query')
df['std_a'] = pd.Series(std_a, dtype='object')

df[["std_ans", "std_cs"]] = df["std_a"].apply(
    lambda x: pd.Series(KG_RAG_Tool.extract_boxed_answer_and_confidence(x))
)

In [ ]:
print(Local.eval_accuracy(df, pred='std_ans', g_t='Label', out_col="is_correct_std"))

std_summary = CalibrationMetrics.summarize(df, prob_col="std_cs", correct_col="is_correct_std", n_bins=10, norm="l2")
print(f'ECE:{std_summary["ece"]}, BS:{std_summary["brier"]}, AUC:{std_summary["selective_auc"]}')

df.to_pickle('standard_rag.pkl')